In [1]:
import sys
sys.version

'3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]'

In [116]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from urllib.parse import urlencode
import urllib3
import string
import json

#### https://csdl-api.computer.org/api/v1/graphql (GraphiQL 주소)

##### Periodicals로 idPrefix, title, yearFrom, yearTo 얻기

In [117]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "query":'query { periodicals(pubType:"journal") { idPrefix title yearFrom yearTo } }'
                      })
body = resp.json()

In [118]:
print(json.dumps(body, indent=4))

{
    "data": {
        "periodicals": [
            {
                "idPrefix": "cq",
                "title": "Colloquium",
                "yearFrom": 2017,
                "yearTo": 2018
            },
            {
                "idPrefix": "dc",
                "title": "IEEE Journal on Exploratory Solid-State Computational Devices and Circuits",
                "yearFrom": 2015,
                "yearTo": 2018
            },
            {
                "idPrefix": "ta",
                "title": "IEEE Transactions on Affective Computing",
                "yearFrom": 2010,
                "yearTo": 2018
            },
            {
                "idPrefix": "bd",
                "title": "IEEE Transactions on Big Data",
                "yearFrom": 2015,
                "yearTo": 2018
            },
            {
                "idPrefix": "cc",
                "title": "IEEE Transactions on Cloud Computing",
                "yearFrom": 2013,
                "yearTo": 2018


In [119]:
journals = [] # (title, idPrefix, yearFrom, yearTo)

for i in range(0,len(body["data"]["periodicals"])):
    journals.append( ( (body["data"]["periodicals"][i]['title']), body["data"]["periodicals"][i]['idPrefix'], body["data"]["periodicals"][i]['yearFrom'], body["data"]["periodicals"][i]['yearTo']) )

In [120]:
journals

[('Colloquium', 'cq', 2017, 2018),
 ('IEEE Journal on Exploratory Solid-State Computational Devices and Circuits',
  'dc',
  2015,
  2018),
 ('IEEE Transactions on Affective Computing', 'ta', 2010, 2018),
 ('IEEE Transactions on Big Data', 'bd', 2015, 2018),
 ('IEEE Transactions on Cloud Computing', 'cc', 2013, 2018),
 ('IEEE Transactions on Computational Intelligence and AI in Games',
  'ci',
  2009,
  2017),
 ('IEEE Transactions on Computers', 'tc', 1968, 2018),
 ('IEEE Transactions on Dependable and Secure Computing', 'tq', 2004, 2018),
 ('IEEE Transactions on Emerging Topics in Computing', 'ec', 2013, 2018),
 ('IEEE Transactions on Haptics', 'th', 2008, 2017),
 ('IEEE Transactions on Knowledge & Data Engineering', 'tk', 1989, 2018),
 ('IEEE Transactions on Learning Technologies', 'lt', 2008, 2017),
 ('IEEE Transactions on Mobile Computing', 'tm', 2002, 2018),
 ('IEEE Transactions on Multi-Scale Computing Systems', 'mc', 2015, 2018),
 ('IEEE Transactions on Network Science and Engin

##### PeriodicalIssues로 year, volume, issueNum 얻기

In [121]:
#예시
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "variables":{"idPrefix":"ci"}, #ci는 하나의 예시임. 실제로는 모든 idPrefix 넣으면서 긁어오기
                          "query":'query ($idPrefix : String!) { periodicalIssues(idPrefix : $idPrefix) { label issueNum volume year } }'
                      })
body = resp.json()

In [122]:
print(json.dumps(body, indent=4))

{
    "data": {
        "periodicalIssues": [
            {
                "label": "March",
                "issueNum": "01",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "June",
                "issueNum": "02",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "Sept.",
                "issueNum": "03",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "Dec.",
                "issueNum": "04",
                "volume": "1",
                "year": "2009"
            },
            {
                "label": "March",
                "issueNum": "01",
                "volume": "2",
                "year": "2010"
            },
            {
                "label": "June",
                "issueNum": "02",
                "volume": "2",
                "year": "2010"
            },
   

In [123]:
for i in range(0, len(journals)) :
    resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                          json={
                              "variables":{"idPrefix":"%s"%(journals[i][1])},
                              "query":'query ($idPrefix : String!) { periodicalIssues(idPrefix : $idPrefix) { label issueNum volume year } }'
                          })
    body = resp.json()
    
    #Todo : 저널 별로 year, volume, issueNum 저장
    print("Progress : %d of %d (%s)" % (i+1, len(journals), journals[i][1]))

Progress : 1 of 26 (cq)
Progress : 2 of 26 (dc)
Progress : 3 of 26 (ta)
Progress : 4 of 26 (bd)
Progress : 5 of 26 (cc)
Progress : 6 of 26 (ci)
Progress : 7 of 26 (tc)
Progress : 8 of 26 (tq)
Progress : 9 of 26 (ec)
Progress : 10 of 26 (th)
Progress : 11 of 26 (tk)
Progress : 12 of 26 (lt)
Progress : 13 of 26 (tm)
Progress : 14 of 26 (mc)
Progress : 15 of 26 (tn)
Progress : 16 of 26 (td)
Progress : 17 of 26 (tp)
Progress : 18 of 26 (sc)
Progress : 19 of 26 (ts)
Progress : 20 of 26 (su)
Progress : 21 of 26 (si)
Progress : 22 of 26 (tg)
Progress : 23 of 26 (tb)
Progress : 24 of 26 (nt)
Progress : 25 of 26 (ca)
Progress : 26 of 26 (lc)


##### articles과 param (idPrefix, year, issueNum)을 통해 article 크롤링

In [124]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "variables":{"idPrefix":"ca", "year":"2018", "issueNum":"02"}, #ci, 2018, 02는 하나의 예시임. 실제로는 idPrefix, year, issueNum 바꿔가며 긁어오기
                          "query":'query ($idPrefix : String!, $year : String!, $issueNum : String!) { articles(idPrefix: $idPrefix, year: $year, issueNum: $issueNum) { idPrefix id year pubDate keywords title abstract authors { affiliation fullName givenName surname } } }'
                      })

In [125]:
body = resp.json()

In [126]:
print(json.dumps(body, indent=4))

{
    "data": {
        "articles": [
            {
                "idPrefix": "ca",
                "id": "13rRUwwslBG",
                "year": "2018",
                "pubDate": "2018-07-01",
                "keywords": [
                    "Nonvolatile Memory",
                    "Random Access Memory",
                    "Encryption",
                    "Computer Architecture",
                    "System On Chip",
                    "Oblivious RAM",
                    "Non Volatile Memory",
                    "Memory Security"
                ],
                "title": "LEO: Low Overhead Encryption ORAM for Non-Volatile Memories",
                "abstract": "Data confidentiality attacks utilizing memory access patterns threaten exposure of data in modern main memories. Oblivious RAM\u00a0(ORAM) is an effective cryptographic primitive developed to thwart access-pattern-based attacks in DRAM-based systems. However, in emerging non-volatile memory\u00a0(NVM) systems, the i

##### articleId로 id, keywords, pubDate, title, year 찾기

In [127]:
resp = requests.post("https://csdl-api.computer.org/api/v1/graphql", 
                      json={
                          "variables":{"articleId":"13rRUwwslBG"},
                          "query":"query ($articleId: String!) { articleById(articleId: $articleId) { id keywords pubDate title year } }"
                      })

In [128]:
body = resp.json()

In [129]:
print(json.dumps(body, indent=4))

{
    "data": {
        "articleById": {
            "id": "13rRUwwslBG",
            "keywords": [
                "Nonvolatile Memory",
                "Random Access Memory",
                "Encryption",
                "Computer Architecture",
                "System On Chip",
                "Oblivious RAM",
                "Non Volatile Memory",
                "Memory Security"
            ],
            "pubDate": "2018-07-01",
            "title": "LEO: Low Overhead Encryption ORAM for Non-Volatile Memories",
            "year": "2018"
        }
    }
}
